In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from datetime import datetime, timedelta
import re
import json
import time
import pandas as pd
from tenacity import retry, stop_after_attempt, wait_fixed, retry_if_exception_type
import logging

In [5]:
TWITTER_AUTH_TOKEN = 'd5f6fb7cebaa271fdd881c395d3c9c0d3f32db7f'

In [6]:


logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


class TwitterExtractor:
    def __init__(self, headless=True):
        self.driver = self._start_chrome(headless)
        self.set_token()

    def _start_chrome(self, headless):
        options = Options()
        options.headless = headless
        driver = webdriver.Chrome(options=options)
        driver.get("https://twitter.com")
        return driver

    def set_token(self, auth_token=TWITTER_AUTH_TOKEN):
        if not auth_token or auth_token == "YOUR_TWITTER_AUTH_TOKEN_HERE":
            raise ValueError("Access token is missing. Please configure it properly.")
        expiration = (datetime.now() + timedelta(days=7)).strftime("%Y-%m-%d")
        cookie_script = f"document.cookie = 'auth_token={auth_token}; expires={expiration}; path=/';"
        self.driver.execute_script(cookie_script)

    def fetch_thread(self, page_url, start_date, end_date):
        self.driver.get(page_url)
        cur_filename = f"data/tweets_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
        print(cur_filename)
        # Convert start_date and end_date from "YYYY-MM-DD" to datetime objects
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
        rows = []
        while True:
            tweet = self._get_first_tweet()
            if not tweet:
                continue

            row = self._process_tweet(tweet)
            print(row)
            if row["date"]:
                try:
                    date = datetime.strptime(row["date"], "%Y-%m-%d")

                except ValueError as e:
                    # infer date format
                    logger.info(
                        f"Value error on date format, trying another format.{row['date']}",
                        e,
                    )
                    date = datetime.strptime(row["date"], "%d/%m/%Y")

                if date < start_date:
                    break
                elif date > end_date:
                    self._delete_first_tweet()
                    continue
            print(row)
            rows.append(row)
            logger.info(
                f"Saving tweets...\n{row['date']},  {row['author_name']} -- {row['text'][:50]}...\n\n"
            )
            self._delete_first_tweet()
        return rows
        

    def fetch_tweets(self, page_url, start_date, end_date):
        self.driver.get(page_url)
        cur_filename = f"data/tweets_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
        print(cur_filename)
        # Convert start_date and end_date from "YYYY-MM-DD" to datetime objects
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
        rows = []
        while True:
            tweet = self._get_first_tweet()
            if not tweet:
                continue
            
            row = self._process_tweet(tweet)
            print(row)
            if row["date"]:
                try:
                    date = datetime.strptime(row["date"], "%Y-%m-%d")

                except ValueError as e:
                    # infer date format
                    logger.info(
                        f"Value error on date format, trying another format.{row['date']}",
                        e,
                    )
                    date = datetime.strptime(row["date"], "%d/%m/%Y")

                if date < start_date:
                    break
                elif date > end_date:
                    self._delete_first_tweet()
                    continue
            print(row)
            rows.append(row)
            logger.info(
                f"Saving tweets...\n{row['date']},  {row['author_name']} -- {row['text'][:50]}...\n\n"
            )
            self._delete_first_tweet()
        return rows

    @retry(
        stop=stop_after_attempt(5),
        wait=wait_fixed(2),
        retry=retry_if_exception_type(TimeoutException),
    )
    def _get_first_tweet(
        self, timeout=10, use_hacky_workaround_for_reloading_issue=True
    ):
        try:
            # Wait for either a tweet or the error message to appear
            WebDriverWait(self.driver, timeout).until(
                lambda d: d.find_elements(By.XPATH, "//article[@data-testid='tweet']")
                or d.find_elements(By.XPATH, "//span[contains(text(),'Try reloading')]")
            )

            # Check for error message and try to click "Retry" if it's present
            error_message = self.driver.find_elements(
                By.XPATH, "//span[contains(text(),'Try reloading')]"
            )
            if error_message and use_hacky_workaround_for_reloading_issue:
                logger.info(
                    "Encountered 'Something went wrong. Try reloading.' error.\nTrying to resolve with a hacky workaround (click on another tab and switch back). Note that this is not optimal.\n"
                )
                logger.info(
                    "You do not have to worry about data duplication though. The save to excel part does the dedup."
                )
                self._navigate_tabs()

                WebDriverWait(self.driver, timeout).until(
                    lambda d: d.find_elements(
                        By.XPATH, "//article[@data-testid='tweet']"
                    )
                )
            elif error_message and not use_hacky_workaround_for_reloading_issue:
                raise TimeoutException(
                    "Error message present. Not using hacky workaround."
                )

            else:
                # If no error message, assume tweet is present
                return self.driver.find_element(
                    By.XPATH, "//article[@data-testid='tweet']"
                )

        except TimeoutException:
            logger.error("Timeout waiting for tweet or after clicking 'Retry'")
            raise
        except NoSuchElementException:
            logger.error("Could not find tweet or 'Retry' button")
            raise

    def _navigate_tabs(self, target_tab="Likes"):
        # Deal with the 'Retry' issue. Not optimal.
        try:
            # Click on the 'Media' tab
            self.driver.find_element(By.XPATH, "//span[text()='Media']").click()
            time.sleep(2)  # Wait for the Media tab to load

            # Click back on the Target tab. If you are fetching posts, you can click on 'Posts' tab
            self.driver.find_element(By.XPATH, f"//span[text()='{target_tab}']").click()
            time.sleep(2)  # Wait for the Likes tab to reload
        except NoSuchElementException as e:
            logger.error("Error navigating tabs: " + str(e))

    @retry(stop=stop_after_attempt(2), wait=wait_fixed(1))
    def _process_tweet(self, tweet):

        author_name, author_handle = self._extract_author_details(tweet)
        try:
            data = {
                "text": self._get_element_text(
                    tweet, ".//div[@data-testid='tweetText']"
                ),
                "author_name": author_name,
                "author_handle": author_handle,
                "date": self._get_element_attribute(tweet, "time", "datetime")[:10],
                "lang": self._get_element_attribute(
                    tweet, "div[data-testid='tweetText']", "lang"
                ),
                "url": self._get_tweet_url(tweet),
                "mentioned_urls": self._get_mentioned_urls(tweet),
                "is_retweet": self.is_retweet(tweet),
                "media_type": self._get_media_type(tweet),
                "images_urls": (
                    self._get_images_urls(tweet)
                    if self._get_media_type(tweet) == "Image"
                    else None
                ),
            }
        except Exception as e:
            logger.error(f"Error processing tweet: {e}")
            logger.info(f"Tweet: {tweet}")
            raise
        # Convert date format
        if data["date"]:
            data["date"] = datetime.strptime(data["date"], "%Y-%m-%d").strftime(
                "%Y-%m-%d"
            )

        # Extract numbers from aria-labels
        data.update(
            {
                "num_reply": self._extract_number_from_aria_label(tweet, "reply"),
                "num_retweet": self._extract_number_from_aria_label(tweet, "retweet"),
                "num_like": self._extract_number_from_aria_label(tweet, "like"),
            }
        )
        return data

    def _get_element_text(self, parent, selector):
        try:
            return parent.find_element(By.XPATH, selector).text
        except NoSuchElementException:
            return ""

    def _get_element_attribute(self, parent, selector, attribute):
        try:
            return parent.find_element(By.CSS_SELECTOR, selector).get_attribute(
                attribute
            )
        except NoSuchElementException:
            return ""

    def _get_mentioned_urls(self, tweet):
        try:
            # Find all 'a' tags that could contain links. You might need to adjust the selector based on actual structure.
            link_elements = tweet.find_elements(
                By.XPATH, ".//a[contains(@href, 'http')]"
            )
            urls = [elem.get_attribute("href") for elem in link_elements]
            return urls
        except NoSuchElementException:
            return []

    def is_retweet(self, tweet):
        try:
            # This is an example; the actual structure might differ.
            retweet_indicator = tweet.find_element(
                By.XPATH, ".//div[contains(text(), 'Retweeted')]"
            )
            if retweet_indicator:
                return True
        except NoSuchElementException:
            return False

    def _get_tweet_url(self, tweet):
        try:
            link_element = tweet.find_element(
                By.XPATH, ".//a[contains(@href, '/status/')]"
            )
            return link_element.get_attribute("href")
        except NoSuchElementException:
            return ""

    def _extract_author_details(self, tweet):
        author_details = self._get_element_text(
            tweet, ".//div[@data-testid='User-Name']"
        )
        # Splitting the string by newline character
        parts = author_details.split("\n")
        if len(parts) >= 2:
            author_name = parts[0]
            author_handle = parts[1]
        else:
            # Fallback in case the format is not as expected
            author_name = author_details
            author_handle = ""

        return author_name, author_handle

    def _get_media_type(self, tweet):
        if tweet.find_elements(By.CSS_SELECTOR, "div[data-testid='videoPlayer']"):
            return "Video"
        if tweet.find_elements(By.CSS_SELECTOR, "div[data-testid='tweetPhoto']"):
            return "Image"
        return "No media"

    def _get_images_urls(self, tweet):
        images_urls = []
        images_elements = tweet.find_elements(
            By.XPATH, ".//div[@data-testid='tweetPhoto']//img"
        )
        for image_element in images_elements:
            images_urls.append(image_element.get_attribute("src"))
        return images_urls

    def _extract_number_from_aria_label(self, tweet, testid):
        try:
            text = tweet.find_element(
                By.CSS_SELECTOR, f"div[data-testid='{testid}']"
            ).get_attribute("aria-label")
            numbers = [int(s) for s in re.findall(r"\b\d+\b", text)]
            return numbers[0] if numbers else 0
        except NoSuchElementException:
            return 0

    def _delete_first_tweet(self, sleep_time_range_ms=(0, 1000)):
        try:
            tweet = self.driver.find_element(
                By.XPATH, "//article[@data-testid='tweet'][1]"
            )
            self.driver.execute_script("arguments[0].remove();", tweet)
        except NoSuchElementException:
            logger.info("Could not find the first tweet to delete.")

    @staticmethod
    def _save_to_json(data, filename="data.json"):
        with open(filename, "a", encoding="utf-8") as file:
            json.dump(data, file)
            file.write("\n")

    @staticmethod
    def _save_to_excel(json_filename, output_filename="data/data.xlsx"):
        # Read JSON data
        cur_df = pd.read_json(json_filename, lines=True)

        # Drop duplicates & save to Excel
        cur_df.drop_duplicates(subset=["url"], inplace=True)
        cur_df.to_excel(output_filename, index=False)
        logger.info(
            f"\n\nDone saving to {output_filename}. Total of {len(cur_df)} unique tweets."
        )




In [7]:
scraper = TwitterExtractor()

In [11]:
tweets = scraper.fetch_tweets(
        "https://x.com/i/lists/1866134598868160880",
        start_date="2024-12-01",
        end_date="2024-12-10",
    ) 

data/tweets_2024-12-10_01-19-51


2024-12-10 01:19:53,709 - __main__ - INFO - Saving tweets...
2024-12-09,  Sam Altman -- amazing work bill!...


2024-12-10 01:19:53,880 - __main__ - INFO - Saving tweets...
2024-12-09,  Sam Altman -- aditya (
@model_mechanic
) is a legend and visiona...




{'text': 'amazing work bill!', 'author_name': 'Sam Altman', 'author_handle': '@sama', 'date': '2024-12-09', 'lang': 'en', 'url': 'https://x.com/sama/status/1866202531090792586', 'mentioned_urls': [], 'is_retweet': False, 'media_type': 'Video', 'images_urls': None, 'num_reply': 0, 'num_retweet': 0, 'num_like': 0}
{'text': 'amazing work bill!', 'author_name': 'Sam Altman', 'author_handle': '@sama', 'date': '2024-12-09', 'lang': 'en', 'url': 'https://x.com/sama/status/1866202531090792586', 'mentioned_urls': [], 'is_retweet': False, 'media_type': 'Video', 'images_urls': None, 'num_reply': 0, 'num_retweet': 0, 'num_like': 0}
{'text': 'aditya (\n@model_mechanic\n) is a legend and visionary in the field, and runs a very special team.', 'author_name': 'Sam Altman', 'author_handle': '@sama', 'date': '2024-12-09', 'lang': 'en', 'url': 'https://x.com/sama/status/1866194899051422077', 'mentioned_urls': ['https://t.co/sBn2oF9a0o', 'https://t.co/sBn2oF9a0o'], 'is_retweet': False, 'media_type': 'No m

2024-12-10 01:19:54,025 - __main__ - INFO - Saving tweets...
2024-12-09,  Sam Altman -- we are launching sora today, and we made a new pro...


2024-12-10 01:19:54,150 - __main__ - INFO - Saving tweets...
2024-12-09,  Sam Altman -- excited to see what you make :)...




{'text': 'we are launching sora today, and we made a new product to go with it.\n\nif you have an openai plus or pro account, you can generate videos. anyone can view them.\n\nit will take some time to roll out, but by the end of the day it should be available at http://sora.com', 'author_name': 'Sam Altman', 'author_handle': '@sama', 'date': '2024-12-09', 'lang': 'en', 'url': 'https://x.com/sama/status/1866187525821538436', 'mentioned_urls': ['https://t.co/VZBcJFqChS'], 'is_retweet': False, 'media_type': 'No media', 'images_urls': None, 'num_reply': 0, 'num_retweet': 0, 'num_like': 0}
{'text': 'we are launching sora today, and we made a new product to go with it.\n\nif you have an openai plus or pro account, you can generate videos. anyone can view them.\n\nit will take some time to roll out, but by the end of the day it should be available at http://sora.com', 'author_name': 'Sam Altman', 'author_handle': '@sama', 'date': '2024-12-09', 'lang': 'en', 'url': 'https://x.com/sama/status/

...

{'text': 'The (true) story of development and inspiration behind the "attention" operator, the one in "Attention is All you Need" that introduced the Transformer. From personal email correspondence with the author \n@DBahdanau\n ~2 years ago, published here and now (with permission) following', 'author_name': 'Andrej Karpathy', 'author_handle': '@karpathy', 'date': '2024-12-03', 'lang': 'en', 'url': 'https://x.com/karpathy/status/1864023344435380613', 'mentioned_urls': [], 'is_retweet': False, 'media_type': 'Image', 'images_urls': ['https://pbs.twimg.com/media/Gd5WAejaQAAVHYh?format=jpg&name=900x900'], 'num_reply': 0, 'num_retweet': 0, 'num_like': 0}
{'text': 'The (true) story of development and inspiration behind the "attention" operator, the one in "Attention is All you Need" that introduced the Transformer. From personal email correspondence with the author \n@DBahdanau\n ~2 years ago, published here and now (with permission) following', 'author_name': 'Andrej Karpathy', 'author_han

2024-12-10 01:19:56,254 - __main__ - INFO - Saving tweets...
2024-12-03,  Andrej Karpathy -- Oh and bleh I forgot to mention for those outside ...


2024-12-10 01:19:56,368 - __main__ - INFO - Saving tweets...
2024-12-01,  Andrej Karpathy -- The reality of the Turing test...


2024-12-10 01:19:56,458 - __main__ - INFO - Saving tweets...
2024-12-01,  Sam Altman -- we made a treehouse!...




{'text': 'Oh and bleh I forgot to mention for those outside AI that ChatGPT (like a lot (most?) of modern AI) is a giant Transformer. So the magic of LLMs at the core comes from a repeated application of Attention, attending over input tokens over and over to predict what token comes next.', 'author_name': 'Andrej Karpathy', 'author_handle': '@karpathy', 'date': '2024-12-03', 'lang': 'en', 'url': 'https://x.com/karpathy/status/1864033537479135369', 'mentioned_urls': [], 'is_retweet': False, 'media_type': 'No media', 'images_urls': None, 'num_reply': 0, 'num_retweet': 0, 'num_like': 0}
{'text': 'Oh and bleh I forgot to mention for those outside AI that ChatGPT (like a lot (most?) of modern AI) is a giant Transformer. So the magic of LLMs at the core comes from a repeated application of Attention, attending over input tokens over and over to predict what token comes next.', 'author_name': 'Andrej Karpathy', 'author_handle': '@karpathy', 'date': '2024-12-03', 'lang': 'en', 'url': 'https:/

2024-12-10 01:19:56,619 - __main__ - INFO - Saving tweets...
2024-12-01,  Sam Altman -- not pictured: both altman brothers backseat drivin...


2024-12-10 01:19:56,701 - __main__ - INFO - Saving tweets...
2024-12-01,  Sam Altman -- but very satisfying to build something physical an...




{'text': 'not pictured: both altman brothers backseat driving and providing almost no help', 'author_name': 'Sam Altman', 'author_handle': '@sama', 'date': '2024-12-01', 'lang': 'en', 'url': 'https://x.com/sama/status/1863015707728089153', 'mentioned_urls': [], 'is_retweet': False, 'media_type': 'No media', 'images_urls': None, 'num_reply': 0, 'num_retweet': 0, 'num_like': 0}
{'text': 'not pictured: both altman brothers backseat driving and providing almost no help', 'author_name': 'Sam Altman', 'author_handle': '@sama', 'date': '2024-12-01', 'lang': 'en', 'url': 'https://x.com/sama/status/1863015707728089153', 'mentioned_urls': [], 'is_retweet': False, 'media_type': 'No media', 'images_urls': None, 'num_reply': 0, 'num_retweet': 0, 'num_like': 0}
{'text': 'but very satisfying to build something physical and better than just eating/drinking all thanksgiving, 10/10 would recommend', 'author_name': 'Sam Altman', 'author_handle': '@sama', 'date': '2024-12-01', 'lang': 'en', 'url': 'https:

In [12]:
tweets

[{'text': 'amazing work bill!',
  'author_name': 'Sam Altman',
  'author_handle': '@sama',
  'date': '2024-12-09',
  'lang': 'en',
  'url': 'https://x.com/sama/status/1866202531090792586',
  'mentioned_urls': [],
  'is_retweet': False,
  'media_type': 'Video',
  'images_urls': None,
  'num_reply': 0,
  'num_retweet': 0,
  'num_like': 0},
 {'text': 'aditya (\n@model_mechanic\n) is a legend and visionary in the field, and runs a very special team.',
  'author_name': 'Sam Altman',
  'author_handle': '@sama',
  'date': '2024-12-09',
  'lang': 'en',
  'url': 'https://x.com/sama/status/1866194899051422077',
  'mentioned_urls': ['https://t.co/sBn2oF9a0o', 'https://t.co/sBn2oF9a0o'],
  'is_retweet': False,
  'media_type': 'No media',
  'images_urls': None,
  'num_reply': 0,
  'num_retweet': 0,
  'num_like': 0},
 {'text': 'we are launching sora today, and we made a new product to go with it.\n\nif you have an openai plus or pro account, you can generate videos. anyone can view them.\n\nit will 